In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import difflib                        # A library that computes the difference between two strings, unused (was for further exploration and info. PS: related to ngrams)

### General functions

In [25]:
def suffix_prefix_detection(s, c):
  """
  Looks up the prefix and suffix of a lemma c. The maximal substring in s that correspond to lemma is extracted
  :params: s for the form and c for the lemma
  :return: the prefix+"-"+suffix, where "-" replaces the inducted substring of the lemma c
  """
  # Perte d'information, quoi faire pour rattaraper ?
  for i in range(len(c),0, -1):
    np = c[:i]
    ns = s.replace(np, '-')
    if ns != s:
      # The substring found, and is replaced
      break
  return ns

def change_learned_accuracies_into_probabilities(corresp):
  """
    A function that changes occurences in corresp to probabilities
    :Example:
    > corresp = {"pre-é": {"V":7, "Noun":3}, "dé-é" : {"V":8, "Adj":2}}
    > probs   = {"pre-é": {"V":0.7, "Noun":0.3}, "dé-é" : {"V":0.8, "Adj":0.2}}
  """
  probs = corresp.copy()
  for key1 in probs.keys():
    sumValues = sum(probs[key1].values())
    for key2 in probs[key1].keys():
      probs[key1][key2] = probs[key1][key2] / sumValues
  return probs

def selectMaxFromDict(dictio):
  """
    Selects the key that correspond to the highest value in a dictionary
  """
  ks = np.array(list(dictio.keys()))
  vs = np.array(list(dictio.values()))
  ord = np.argsort(vs)
  ks_n = ks[ord]
  return ks_n[0] if not ks_n.shape[0] == 0 else '-'

def selectFromDict(dictio, threshold):
  """
    Selects cases (keys) from dictionary where the values are >= threshold. Those are returned ordered from highest to lowest.
  """
  ks = np.array(list(dictio.keys()))
  vs = np.array(list(dictio.values()))
  ks = ks[vs>=threshold]
  vs = vs[vs>=threshold]
  ord = np.argsort(vs)
  ks_n = ks[ord]
  return ks_n

def CharByCharAccuracy(lemma1, lemma2):
  """
    An example to defining the distance between two lemmas
  """
  d = suffix_prefix_detection(lemma1, lemma2)
  d = d.replace('-', '')
  return np.mean(list(map(lambda a:1,d))+[0])/max(len(lemma1), len(lemma2))

### Functions exclusively for the naïve version

In [28]:
def best_candidates_from_probabilities_naive(morphs, probs):
  """
  An approach that searches for potential candidates for the changes.
  It simply regroupes the different changes that have been recorded to have had *the exact same* morphological attributes
  :params:  * morphs: (String) morphological attributes, mainly on the test data set
            * Probs: A dictionary that regroupe changes to their morphological attributes and there occurences on training dataset
  :returns: A dictionnary with potential changes to make to a lemma, and the corresponding occurence within the training dataset
  """
  possible_predictions = dict()
  for c in probs.keys():
    dict_probs = probs[c]
    if set(morphs.split(";")) == set(dict_probs.keys()):
      if c in possible_predictions.keys():
        possible_predictions[c] += 1
      else:
        possible_predictions[c] = 1
    # If an example is not met on the training set, we abandon the cause. Hence, obvious
  return possible_predictions


def train_naive(fd):
  """
    Simple function creating the tuple (lemma, form, M.A) also a dictionnary corresp.
    :params: fd is a file descriptor
    :return corresp: (dict) The changes "prefix-suffix", the attributes and occurences that correspond to them.
  """
  # Create words
  l = fd.readline()
  ws, fs, rs = list(), list(), list()
  while l:
      w, f, r = list( map(lambda a : a.strip(), l.split('\t')) )
      ws.append(w); fs.append(f); rs.append(r)
      l = fd.readline()
  # Create a dictionnary with rule (pre-suf) as keys and have the corresponding morph attributes 
  corresp = dict()
  for i in range(len(ws)):
    rule = suffix_prefix_detection(fs[i], ws[i])
    if rule in corresp.keys():
      for k in rs[i].split(";"):
        if k in corresp[rule]:
          corresp[rule][k] += 1
        else:
          corresp[rule][k] = 1
    else:
        corresp[rule] =  {key: 1 for key in rs[i].split(";")}
  return (ws, fs, rs), corresp

def printAccuracies_naive(fdTrain, fdTest):
  """
	Prints accuracies in the case where the test set disposes actually of forms that are to be predicted
  """
  # Training data and dependencies
  (ws, fs, rs), corresp = train_naive(fdTrain)
  print("Number of the training examples are ", len(ws))
  probs = change_learned_accuracies_into_probabilities(corresp)
  # Test data
  (ws_t, fs_t, rs_t), _ = train_naive(fdTest)
  print("Number of the testing examples are =", len(ws_t))
  acc, acc2 = list(), list()
  # For each test sample
  for i in range(len(rs_t)):
    c = selectMaxFromDict(best_candidates_from_probabilities_naive(rs_t[i], probs))
    # Without taking the lemma to account
    c = c.replace("-", ws_t[i])
    cbc = CharByCharAccuracy(c, fs_t[i])
    acc.append(cbc)
    acc2.append(fs_t[i] == c)
  print("Char-by-char accuracy is:", 1 - np.mean(acc), ". Word-to-word on the other hand is:", np.mean(acc2))

### Advanced approach methods

In [4]:
def best_candidates_from_probabilities(morphs, probs):
  """Same as before, but records the cases where only one of the morphological attributes is present"""
  possible_predictions = dict()
  for c in probs.keys():
    dict_probs = probs[c]
    if np.all(list(map(lambda a : a in dict_probs.keys(), morphs.split(";")))):
      # This is the case where the training set has a sample that had the exacte Morph. Att. and is amplified (+2)
      if c in possible_predictions.keys():
        possible_predictions[c] += 2
      else:
        possible_predictions[c] = 2
    # The case where at least one morph. att. is present. This case is less representable (+1) 
    elif np.any(list(map(lambda a : a in dict_probs.keys(), morphs.split(";")))):
      if c in possible_predictions.keys():
        possible_predictions[c] += 1
      else:
        possible_predictions[c] = 1
    # +0 for the case of no presence. This occurences and amplification serve for the probabilistic model later on.
  return possible_predictions

def best_candidates_from_probabilities_advanced(lemma, morphs, corresp, corresp2, corresp3, dist):
  """
    This is an advanced method. It takes to account some learned information from the training set to 
    select a potential candidates. It, all the same, is still a naïve approach.
    :params:  * corresp: Contains, as probs before, the changes "prefix-suffix", the attributes and occurences that correspond to them.
              * corresp2: Contains changes and the corresponding lemmas.
              * corresp3: Contains the changes and their occurences within the training data set (the world)
              * dist: Distance to defines as one pleases between two lemmas
    :return: The most probable changes taking to account:
    * Their M.A. and those of the test set
    * The probability that these changes occur (if they accur rarely in the training set, so they should in the test set)
    * The distance of the new lemma from the lemmas in the training set (if this lemma is so close to another one, so the changes are 
    most probably the same as those on the training set)
  """
  pp = best_candidates_from_probabilities(morphs, change_learned_accuracies_into_probabilities(corresp))
  new_preds = dict.fromkeys(pp.keys())
  for k in pp.keys():
    dists = list(map(lambda a: dist(a, lemma),corresp3[k]))
    new_preds[k] = pp[k] * corresp2[k] * (np.max(dists) - np.mean(dists))
  minIdx = np.argmax(list(new_preds.values()))
  return np.array(list(new_preds.keys()))[minIdx]

def train(fd):
  """
    Simple function creating the tuple (lemma, form, M.A) also the tuple (corresp, corresp2, corresp3) as:
              * corresp: The changes "prefix-suffix", the attributes and occurences that correspond to them.
              * corresp2: Contains changes and the corresponding lemmas.
              * corresp3: Contains the changes and their occurences within the training data set (the world)
    :params: fd is a file descriptor
  """
  # Create words
  l = fd.readline()
  ws, fs, rs = list(), list(), list()
  while l:
      w, f, r = list( map(lambda a : a.strip(), l.split('\t')) )
      ws.append(w); fs.append(f); rs.append(r)
      l = fd.readline()
  # Create a dictionnary with rule (pre-suf) as keys and have the corresponding morph attributes 
  corresp = dict()
  corresp2 = dict()
  corresp3 = dict()
  for i in range(len(ws)):
    rule = suffix_prefix_detection(fs[i], ws[i])
    if rule in corresp.keys():
      corresp2[rule] += 1
      corresp3[rule].append(ws[i])
      for k in rs[i].split(";"):
        if k in corresp[rule]:
          corresp[rule][k] += 1
        else:
          corresp[rule][k] = 1
    else:
        corresp[rule] =  {key: 1 for key in rs[i].split(";")}
        corresp2[rule] = 1
        corresp3[rule] = [ws[i]]
  return (ws, fs, rs), (corresp, corresp2, corresp3)

In [5]:
def distanceFromLemma(lemma1, lemma2):
  """
    An example to defining the distance between two lemmas
  """
  d = 0
  # Distance is the sum of the character-wise differences
  for i in range(min(len(lemma1), len(lemma2))):
    d += abs(ord(lemma1[i]) - ord(lemma2[i]))
  lemma = lemma1 if len(lemma1) > len(lemma2) else lemma2
  # Adding the difference
  reste = sum(list(map(lambda a : ord(a),lemma[i+1:])))
  return d + reste

def distanceFromLemma_(lemma1, lemma2):
  """
    Another example where first and last character are more put to interest than the reste. And no character-wise distance.
  """
  b = 0.5 * (lemma1 == lemma2) + 0.25 * ((lemma1[:2] == lemma2[:2]) + (lemma1[-2:] == lemma2[-2:])) if min(len(lemma1), len(lemma2)) >= 2 else int(lemma1 == lemma2)
  return b

In [27]:
def printAccuracies(fdTrain, fdTest):
  """
	Prints accuracies in the case where the test set disposes actually of forms that are to be predicted
  """
  # Training data and dependencies
  (ws, fs, rs), (corresp, corresp2, corresp3) = train(fdTrain)
  print("Number of the training examples are ", len(ws))
  probs = change_learned_accuracies_into_probabilities(corresp)
  # Test data
  (ws_t, fs_t, rs_t), _ = train(fdTest)
  print("Number of the testing examples are =", len(ws_t))
  acc, acc2 = list(), list()
  # Defining a distance that is a sum of the above mentionned ones
  dist = lambda a, b : distanceFromLemma(a, b) - distanceFromLemma_(a, b)
  # For each test sample
  for i in range(len(rs_t)):
    c = best_candidates_from_probabilities_advanced(ws_t[i], rs_t[i], corresp, corresp2, corresp3, dist)
    # Without taking the lemma to account
    c = c.replace("-", ws_t[i])
    cbc = CharByCharAccuracy(c, fs_t[i])
    acc.append(cbc)
    acc2.append(fs_t[i] == c)
    
  print("Char-by-char accuracy is:", 1 - np.mean(acc), ". Word-to-word on the other hand is:", np.mean(acc2))

### Demonstrations

In [30]:
fdTrain = open("swa.trn")
fdTest = open("swa.tst")
print("For Swahli naïve")
printAccuracies_naive(fdTrain, fdTest)
print("For Swahli advanced")
fdTrain = open("swa.trn")
fdTest = open("swa.tst")
printAccuracies(fdTrain, fdTest)
fdTrain = open("hil.trn")
fdTest = open("hil.tst")
print("For Hil naïve")
printAccuracies_naive(fdTrain, fdTest)
fdTrain = open("hil.trn")
fdTest = open("hil.tst")
print("For Hil advanced")
printAccuracies(fdTrain, fdTest)

For Swahli naïve
Number of the training examples are  3374
Number of the testing examples are = 910
Char-by-char accuracy is: 1.0 . Word-to-word on the other hand is: 1.0
For Swahli advanced
Number of the training examples are  3374
Number of the testing examples are = 910
Char-by-char accuracy is: 0.9927754772699827 . Word-to-word on the other hand is: 0.9263736263736264
For Hil naïve
Number of the training examples are  859
Number of the testing examples are = 238
Char-by-char accuracy is: 0.9594787565649753 . Word-to-word on the other hand is: 0.46638655462184875
For Hil advanced
Number of the training examples are  859
Number of the testing examples are = 238
Char-by-char accuracy is: 0.9290535123003405 . Word-to-word on the other hand is: 0.08823529411764706


In [32]:
fdTrain = open("mlg.trn")
fdTest = open("mlg.tst")
print("For MLG naïve")
printAccuracies_naive(fdTrain, fdTest)
print("For MLG advanced")
fdTrain = open("mlg.trn")
fdTest = open("mlg.tst")
printAccuracies(fdTrain, fdTest)

fdTrain = open("lug.trn")
fdTest = open("lug.tst")
print("For LUG naïve")
printAccuracies_naive(fdTrain, fdTest)
print("For LUG advanced")
fdTrain = open("lug.trn")
fdTest = open("lug.tst")
printAccuracies(fdTrain, fdTest)

For MLG naïve
Number of the training examples are  447
Number of the testing examples are = 127
Char-by-char accuracy is: 1.0 . Word-to-word on the other hand is: 1.0
For MLG advanced
Number of the training examples are  447
Number of the testing examples are = 127
Char-by-char accuracy is: 0.9979241231209736 . Word-to-word on the other hand is: 0.9763779527559056
For LUG naïve
Number of the training examples are  3420
Number of the testing examples are = 977
Char-by-char accuracy is: 0.9596160346407826 . Word-to-word on the other hand is: 0.49437052200614123
For LUG advanced
Number of the training examples are  3420
Number of the testing examples are = 977
Char-by-char accuracy is: 0.9450548821756142 . Word-to-word on the other hand is: 0.3940634595701126


In [ ]:
fdTrain = open("isl.trn")
fdTest = open("isl.tst")
print("For ISL naïve")
printAccuracies_naive(fdTrain, fdTest)
print("For ISL advanced")
fdTrain = open("isl.trn")
fdTest = open("isl.tst")
printAccuracies(fdTrain, fdTest)

fdTrain = open("krl.trn")
fdTest = open("krl.tst")
print("For KRL naïve")
printAccuracies_naive(fdTrain, fdTest)
print("For KRL advanced")
fdTrain = open("krl.trn")
fdTest = open("krl.tst")
printAccuracies(fdTrain, fdTest)

For ISL naïve
Number of the training examples are  53841
Number of the testing examples are = 15384
Char-by-char accuracy is: 0.9276864037013997 . Word-to-word on the other hand is: 0.06415756630265211
For ISL advanced
Number of the training examples are  53841
Number of the testing examples are = 15384


### Unused functions

In [ ]:
def look_changes(morphs, probs, threshold):
  """
  A very naïve approach that searches for potential candidates for the changes.
  It simply regroupes the different changes that have been recorded to have had one amongst morphological attributes
  :params:  * morphs: (array) morphological attributes, mainly on the test data set
            * Probs: A dictionary that regroupe changes to their morphological attributes and there occurences on training dataset
            * Threshold: Unused, but serves as an indication to ignore underrated examples
  :returns: A dictionnary with potential changes to make to a lemma, and the corresponding accumulative probabilities
  within the training dataset
  """
  possible_predictions = dict()
  for c in probs.keys():
    dict_probs = probs[c]
    for morph in morphs:
      if morph in dict_probs.keys() and dict_probs[morph] >= threshold:
        if c in possible_predictions.keys():
          possible_predictions[c] += dict_probs[morph]
        else:
          possible_predictions[c] = dict_probs[morph]
  return possible_predictions

def generate_test(fd):
  """
    In the case of test datasets, the tuple (lemma, M.A) only is returned. Not used as we are in demo mode ;)
  """
  i = 0
  l = fd.readline()
  ws, rs = list(), list()
  while l:
      w, r = list( map(lambda a : a.strip(), l.split('\t')) )
      ws.append(w); rs.append(r)
      l = fd.readline()
      i += 1
      #if i > 700:
        #break
  return (ws, rs)

def lemmasAndChanges(ws, fs, rs):
  """
    From (lemma, form, M.A.) return (lemma, changes, M.A.)
  """
  cs = list(map(lambda f, w: suffix_prefix_detection(f, w), zip(fs, ws)))
  return ws, cs, rs